<h1>Battle of Neighbourhood | Finding out the best possible location for buying or renting home in Kolkata

In [25]:
import numpy as np
import pandas as pd

import json
!pip install bs4
from bs4 import BeautifulSoup as bs
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize
!pip install geocoder
import geocoder

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium
import requests
print('Libraries imported.')

Libraries imported.


In [26]:
lat=[]
lng=[]
nei=[]

**Fetching data of neighbourhood,latitude and longitude from wikipedia**

In [27]:
for i in range(1,145):
  df=pd.read_html('https://en.wikipedia.org/wiki/Ward_No._'+str(i)+',_Kolkata_Municipal_Corporation')
  df=df[0]
  st=df.loc[2]['Ward No. '+str(i)]
  s=st.split()
  latnew=s[-2].replace('\ufeff','').split('°')[0]
  lngnew=s[-1].split('°')[0]
  neinew=df.loc[6]['Ward No. '+str(i)+'.1']
  lat.append(float(latnew))
  lng.append(float(lngnew))
  nei.append(neinew)

In [28]:
len(nei)

144

**Assigning the data to our dataframe**

In [29]:
df = pd.DataFrame({'Ward no':[x for x in range(1,145)], 
                    'Neighbourhood':nei, 
                    'Latitude':lat,
                   'Longitude':lng
                   })

**Setting ward no as our index**

In [30]:
df.set_index('Ward no',inplace=True)

In [31]:
df.head(15)

,Neighbourhood,Latitude,Longitude
Ward no,,,
1,Cossipore,22.617889,88.370556
2,Sinthee (Ramlila Bagan-Biswanath Colony-Roypar...,22.628056,88.384444
3,"Belgachia, Duttabagan",22.604444,88.383333
4,Paikpara,22.613056,88.379444
5,"Tala, Belgachia",22.608889,88.379694
6,"Chitpur, Cossipore",22.610863,88.371213
7,Bagbazar,22.603567,88.365806
8,"Bagbazar, Shobhabazar",22.601806,88.366500
9,"Shobhabazar, Kumortuli",22.595889,88.365306


In [32]:
df.describe()

,Latitude,Longitude
count,144.000000,144.000000
mean,22.539073,88.352142
std,0.042767,0.031927
min,22.456589,88.254556
25%,22.504333,88.336507
50%,22.540771,88.360938
75%,22.576410,88.372452
max,22.628056,88.409694


**Plotting the wards data in map**

In [33]:
kol_wards = folium.Map(location=[22.5726, 88.3639], zoom_start=11)
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(kol_wards)
kol_wards

<h2>Now fetching nearby venue data for each ward</h2> 

In [34]:

CLIENT_ID = 'JCE2JSWQW13EM4EQ4STWA1QTUUECTHTXDNIPWDPQEEQXL0QZ' # Put Your Client Id
CLIENT_SECRET = 'S0UYS3P0KGCP5ZG1XMYSX2GDSQZ0MTROPDUIN3O5XR2URQDX' # Put You Client Secret 
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [36]:
kolkata_venues = getNearbyVenues(names=df['Neighbourhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])

Cossipore
Sinthee (Ramlila Bagan-Biswanath Colony-Roypara), Sobji Bagan, Satpukur
Belgachia, Duttabagan
Paikpara
Tala, Belgachia
Chitpur, Cossipore
Bagbazar
Bagbazar, Shobhabazar
Shobhabazar, Kumortuli
Shyambazar, Shobhabazar, Shyampukur, Hatibagan
Shyambazar, Burtolla, Hatibagan
Shyambazar (Fariapukur)
Ultadanga (Daspara-Muchi Bazar-Telenga Bagan-Surir Bagan)
Ultadanga (Murari Pukur), Bagmari
Maniktala (Bangiya Sahitya Parishad-Halsi Bagan-Gouribari)
Maniktala (Bangiya Sahitya Parishad-Goa Bagan)
Darjipara
Shobhabazar, Beniatola (Garanhatta)
Shobhabazar, Beniatola (Ahiritola)
Beniatola (Ahiritola)
Jorabagan, Malapara
Posta
Posta, Jorasanko
Jorabagan, Pathuriaghata
Rambagan, Girish Park, Jorasanko, Simla
Ramdulal Sarkar Street and Hedua Park
Maniktala
Maniktala, Garpar, Rajabazar
Narkeldanga
Narkeldanga, Phoolbagan, Kankurgachi
Kankurgachi, Phoolbagan (Kadapara)
Kankurgachi, Bagmari, Ultadanga (Murari Pukur-Sreekrishna Colony)
Phoolbagan (Subhas Sarovar), Beliaghata (Kulia)
Beliaghata 

<h3>Got our venue data in our Dataframe </h3>

In [37]:
kolkata_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cossipore,22.617889,88.370556,Golbari,22.601742,88.373927,Indian Restaurant
1,Cossipore,22.617889,88.370556,Kotak Mahindra Bank - Branch/ATM,22.603536,88.369274,Bank
2,Cossipore,22.617889,88.370556,Five Point Crossing,22.601878,88.373793,Plaza
3,Cossipore,22.617889,88.370556,Sinthee More,22.630965,88.378465,Plaza
4,Cossipore,22.617889,88.370556,Baghbazar Ghat,22.604887,88.364826,River


In [38]:
kolkata_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alipore,32,32,32,32,32,32
"Ashok Nagar, Kudghat, Tollygunge Club, Regent Park (Shantigarh-Regent Colony-Nehru Colony-Vivekananda Nagar-Moor Avenue-Shanti Nagar-Composite Housing Estate)",18,18,18,18,18,18
"Badartala, Rajabagan",5,5,5,5,5,5
Bagbazar,16,16,16,16,16,16
"Bagbazar, Shobhabazar",16,16,16,16,16,16
...,...,...,...,...,...,...
Tollygunge Circular Road (Sirity-Senhati Colony),13,13,13,13,13,13
Ultadanga (Daspara-Muchi Bazar-Telenga Bagan-Surir Bagan),26,26,26,26,26,26
"Ultadanga (Murari Pukur), Bagmari",33,33,33,33,33,33


In [39]:
print('There are {} uniques categories.'.format(len(kolkata_venues['Venue Category'].unique())))

There are 125 uniques categories.


<h3>Now getting dummy variables for the categorical venues for our unsupervised clusturing</h3>

In [40]:
kolkata_onehot = pd.get_dummies(kolkata_venues[['Venue Category']], prefix="", prefix_sep="")
kolkata_onehot['Neighborhood'] = kolkata_venues['Neighborhood'] 

fixed_columns = [kolkata_onehot.columns[-1]] + list(kolkata_onehot.columns[:-1])
kolkata_onehot = kolkata_onehot[fixed_columns]

kolkata_onehot.head()

,Zoo,ATM,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Awadhi Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Cricket Ground,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,...,Music Venue,Neighborhood,Nightclub,North Indian Restaurant,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Plaza,Pool,Port,Pub,Racetrack,Residential Building (Apartment / Condo),Restaurant,River,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sports Club,Stadium,Steakhouse,Supermarket,Taxi Stand,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Restaurant,Tibetan Restaurant,Train Station,Tram Station,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,Cossipore,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,Cossipore,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,Cossipore,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,Cossipore,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,Cossipore,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<h3>Now grouping our data with the mean of frequency

In [41]:
kolkata_grouped = kolkata_onehot.groupby('Neighborhood').mean().reset_index()
kolkata_grouped

,Neighborhood,Zoo,ATM,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Awadhi Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Cricket Ground,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,...,Museum,Music Venue,Nightclub,North Indian Restaurant,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Plaza,Pool,Port,Pub,Racetrack,Residential Building (Apartment / Condo),Restaurant,River,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sports Club,Stadium,Steakhouse,Supermarket,Taxi Stand,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Restaurant,Tibetan Restaurant,Train Station,Tram Station,Vegetarian / Vegan Restaurant
0,Alipore,0.031250,0.000000,0.0,0.0,0.0,0.0,0.031250,0.0,0.0,0.031250,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.125000,0.062500,0.0,0.000000,0.0,0.0,0.000000,0.03125,0.093750,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0000,0.062500,0.0,0.0,0.0,0.062500,0.0,0.0,0.031250,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.031250,0.03125,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.031250,0.000000
1,"Ashok Nagar, Kudghat, Tollygunge Club, Regent ...",0.000000,0.222222,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.111111,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.111111,...,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.111111,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.00000,0.055556,0.0,0.0,0.055556,0.0,0.000000,0.055556,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2,"Badartala, Rajabagan",0.000000,0.800000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.200000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3,Bagbazar,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.062500,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.062500,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0000,0.125000,0.0,0.0,0.0,0.062500,0.0,0.0,0.062500,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.062500,0.000000,0.000000
4,"Bagbazar, Shobhabazar",0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.062500,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.062500,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0000,0.125000,0.0,0.0,0.0,0.062500,0.0,0.0,0.062500,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.062500,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Tollygunge Circular Road (Sirity-Senhati Colony),0.000000,0.230769,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.076923,0.

In [42]:
kolkata_grouped.shape

(142, 125)

<h3>printing top 5 venues in each ward

In [43]:
num_top_venues = 5
for neigh in kolkata_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = kolkata_grouped[kolkata_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alipore----
                  venue  freq
0                  Café  0.12
1                 Dhaba  0.09
2     Indian Restaurant  0.06
3           Pizza Place  0.06
4  Fast Food Restaurant  0.06


----Ashok Nagar, Kudghat, Tollygunge Club, Regent Park (Shantigarh-Regent Colony-Nehru Colony-Vivekananda Nagar-Moor Avenue-Shanti Nagar-Composite Housing Estate)----
                 venue  freq
0                  ATM  0.22
1  Dumpling Restaurant  0.11
2                Plaza  0.11
3                 Café  0.11
4               Lounge  0.06


----Badartala, Rajabagan----
           venue  freq
0            ATM   0.8
1         Bakery   0.2
2  Military Base   0.0
3           Pier   0.0
4       Pharmacy   0.0


----Bagbazar----
                venue  freq
0       Metro Station  0.12
1                Park  0.12
2   Indian Restaurant  0.06
3  Bengali Restaurant  0.06
4              Market  0.06


----Bagbazar, Shobhabazar----
                venue  freq
0       Metro Station  0.12
1                

<h3>Now saving the top 5 venue data in dataframe

In [44]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = kolkata_grouped['Neighborhood']

for ind in np.arange(kolkata_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kolkata_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(142, 11)

<h3>
Creating KMeans clustering

In [46]:
kclusters = 5
kolkata_grouped_clustering = kolkata_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(kolkata_grouped_clustering)
print(kmeans.labels_)

[0 4 1 3 3 2 0 0 0 0 0 0 0 0 2 0 4 0 3 0 0 3 0 4 0 3 3 0 0 0 0 3 0 0 0 0 0
 0 0 0 0 3 3 3 0 3 0 0 0 2 3 0 0 0 0 0 0 1 3 1 4 3 0 2 2 2 0 2 0 3 0 0 0 4
 3 3 3 0 0 0 0 0 0 0 3 4 0 3 3 3 0 3 4 1 1 0 0 0 2 0 0 0 1 3 0 3 3 3 3 3 0
 0 0 4 1 0 0 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 4 3 3 0 3]


In [47]:
len(kmeans.labels_)

142

In [48]:
df.head()

,Neighbourhood,Latitude,Longitude
Ward no,,,
1,Cossipore,22.617889,88.370556
2,Sinthee (Ramlila Bagan-Biswanath Colony-Roypar...,22.628056,88.384444
3,"Belgachia, Duttabagan",22.604444,88.383333
4,Paikpara,22.613056,88.379444
5,"Tala, Belgachia",22.608889,88.379694


<h3>
Merging the clustured label data with our dataframe

In [49]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [50]:
kolkata_merged = df
kolkata_merged = kolkata_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
kolkata_merged.head()

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Ward no,,,,,,,,,,,,,,
1,Cossipore,22.617889,88.370556,2,Plaza,Vegetarian / Vegan Restaurant,Train Station,River,Indian Restaurant,Bank,Metro Station,Dumpling Restaurant,Fast Food Restaurant,Falafel Restaurant
2,Sinthee (Ramlila Bagan-Biswanath Colony-Roypar...,22.628056,88.384444,3,Pizza Place,Platform,Plaza,Bakery,Donut Shop,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dumpling Restaurant,Dhaba
3,"Belgachia, Duttabagan",22.604444,88.383333,3,Metro Station,Train Station,Indie Movie Theater,Bengali Restaurant,Mughlai Restaurant,Plaza,Café,Market,Indian Sweet Shop,Indian Restaurant
4,Paikpara,22.613056,88.379444,2,Metro Station,Train Station,Plaza,Mughlai Restaurant,Indian Restaurant,Platform,Bakery,Fast Food Restaurant,Falafel Restaurant,Electronics Store
5,"Tala, Belgachia",22.608889,88.379694,3,Train Station,Metro Station,Bengali Restaurant,Indian Restaurant,Indian Sweet Shop,Asian Restaurant,Indie Movie Theater,Mughlai Restaurant,Platform,Bakery



<h3>Finally creating our Kolkta map with clustured data

In [51]:
latitude=22.5726
longitude=88.3639
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(kolkata_merged['Latitude'], kolkata_merged['Longitude'], kolkata_merged['Neighbourhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters

<h3>Analyzing cluster 0 </h3>

In [52]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 0, kolkata_merged.columns[[0] + list(range(4, kolkata_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Ward no,,,,,,,,,,,
28,"Maniktala, Garpar, Rajabazar",Café,Department Store,Train Station,Park,Pizza Place,Indian Sweet Shop,Juice Bar,Dhaba,Mughlai Restaurant,Coffee Shop
29,Narkeldanga,Café,Train Station,Department Store,Park,Pizza Place,Bus Stop,Platform,Bengali Restaurant,Bus Station,Juice Bar
30,"Narkeldanga, Phoolbagan, Kankurgachi",Café,Indian Restaurant,Fast Food Restaurant,Train Station,Multiplex,Pizza Place,Bookstore,Department Store,Shopping Mall,Bus Station
31,"Kankurgachi, Phoolbagan (Kadapara)",Café,Indian Restaurant,Fast Food Restaurant,Multiplex,Vegetarian / Vegan Restaurant,Shopping Mall,History Museum,Italian Restaurant,Fried Chicken Joint,Dhaba
32,"Kankurgachi, Bagmari, Ultadanga (Murari Pukur-...",Café,Indian Restaurant,Multiplex,Fast Food Restaurant,Pizza Place,Shopping Mall,Chinese Restaurant,Coffee Shop,Bakery,Bookstore
...,...,...,...,...,...,...,...,...,...,...,...
120,Behala (James Long Sarani-Bliss Park-Behala Ga...,Mughlai Restaurant,Ice Cream Shop,ATM,Indian Sweet Shop,Dhaba,Café,Health & Beauty Service,Movie Theater,Electronics Store,Fast Food Restaurant
123,Barisha,Café,ATM,Indian Restaurant,Indie Movie Theater,Athletics & Sports,Bar,Falafel Restaurant,Food,Flea Market,Field
128,Behala (Ramnarayan Government Colony-Dhamarajt...,Indie Movie Theater,Café,Market,Health & Beauty Service,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store


<h3>Analyzing Cluster 1</h3>

In [53]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 1, kolkata_merged.columns[[0] + list(range(4, kolkata_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Ward no,,,,,,,,,,,
125,"Paschim Barisha, Thakurpukur",ATM,Jewelry Store,Bakery,Vegetarian / Vegan Restaurant,Flea Market,Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dumpling Restaurant
126,"Sarsuna, Paschim Barisha",ATM,Jewelry Store,Pharmacy,Bakery,Vegetarian / Vegan Restaurant,Flea Market,Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store
133,Garden Reach (Akshay Kanan),ATM,Pizza Place,Harbor / Marina,Pier,Construction & Landscaping,Cricket Ground,Department Store,Dessert Shop,Dhaba,Flea Market
136,Garden Reach (Metiabruz),ATM,Pizza Place,River,Construction & Landscaping,Hookah Bar,History Museum,Cricket Ground,Department Store,Dessert Shop,Dhaba
138,Metiabruz (Panchpara-Kashyappara-Jabarhat),ATM,River,Bakery,Vegetarian / Vegan Restaurant,Electronics Store,Flea Market,Field,Fast Food Restaurant,Falafel Restaurant,Dumpling Restaurant
140,Metiabruz (Kanthalberia-Mullickpara),ATM,River,Bakery,Vegetarian / Vegan Restaurant,Electronics Store,Flea Market,Field,Fast Food Restaurant,Falafel Restaurant,Dumpling Restaurant
141,"Badartala, Rajabagan",ATM,Bakery,Vegetarian / Vegan Restaurant,Electronics Store,Food,Flea Market,Field,Fast Food Restaurant,Falafel Restaurant,Dumpling Restaurant


<h3>Analyzing Cluster 2</h3>

In [54]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 2, kolkata_merged.columns[[0] + list(range(4, kolkata_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Ward no,,,,,,,,,,,
1,Cossipore,Plaza,Vegetarian / Vegan Restaurant,Train Station,River,Indian Restaurant,Bank,Metro Station,Dumpling Restaurant,Fast Food Restaurant,Falafel Restaurant
4,Paikpara,Metro Station,Train Station,Plaza,Mughlai Restaurant,Indian Restaurant,Platform,Bakery,Fast Food Restaurant,Falafel Restaurant,Electronics Store
100,Garia (Ramgarh-Naktala-Laxmi Narayan Colony),Metro Station,Café,Plaza,Dumpling Restaurant,Construction & Landscaping,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop
101,"Baghajatin (Ajanta Park), Garia (Phool Bagan-G...",Metro Station,Café,Plaza,Multiplex,Shopping Mall,Vegetarian / Vegan Restaurant,Department Store,Dessert Shop,Dhaba,Diner
109,"Haltu (Kalikapur), Santoshpur (Borakhola-Nanda...",Metro Station,Grocery Store,Multiplex,Shopping Mall,Flea Market,Cricket Ground,Department Store,Dessert Shop,Dhaba,Diner
110,Garia (Baishnabghata Patuli Township-Kanungo P...,Metro Station,Café,Plaza,Multiplex,Shopping Mall,Vegetarian / Vegan Restaurant,Department Store,Dessert Shop,Dhaba,Diner
111,Garia–Brahmapur–Kamdahari-Boral,Metro Station,Café,Plaza,Bus Station,Food,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop
112,Bansdroni (Rania-Roy Nagar),Metro Station,Supermarket,Café,Plaza,Vegetarian / Vegan Restaurant,Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dumpling Restaurant


<h3>Analyzing Cluster 3</h3>

In [55]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 3, kolkata_merged.columns[[0] + list(range(4, kolkata_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Ward no,,,,,,,,,,,
2,Sinthee (Ramlila Bagan-Biswanath Colony-Roypar...,Pizza Place,Platform,Plaza,Bakery,Donut Shop,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dumpling Restaurant,Dhaba
3,"Belgachia, Duttabagan",Metro Station,Train Station,Indie Movie Theater,Bengali Restaurant,Mughlai Restaurant,Plaza,Café,Market,Indian Sweet Shop,Indian Restaurant
5,"Tala, Belgachia",Train Station,Metro Station,Bengali Restaurant,Indian Restaurant,Indian Sweet Shop,Asian Restaurant,Indie Movie Theater,Mughlai Restaurant,Platform,Bakery
6,"Chitpur, Cossipore",Market,Indian Sweet Shop,Metro Station,Fast Food Restaurant,Mughlai Restaurant,Plaza,Indie Movie Theater,Bengali Restaurant,Indian Restaurant,Train Station
7,Bagbazar,Park,Metro Station,Indie Movie Theater,Pizza Place,Café,Plaza,Mughlai Restaurant,Fast Food Restaurant,Market,Bengali Restaurant
8,"Bagbazar, Shobhabazar",Park,Metro Station,Indie Movie Theater,Pizza Place,Café,Plaza,Mughlai Restaurant,Fast Food Restaurant,Market,Bengali Restaurant
9,"Shobhabazar, Kumortuli",Park,History Museum,Metro Station,Fast Food Restaurant,Pizza Place,Mughlai Restaurant,Plaza,Café,Bakery,Indie Movie Theater
10,"Shyambazar, Shobhabazar, Shyampukur, Hatibagan",Park,History Museum,Metro Station,Fast Food Restaurant,Pizza Place,Mughlai Restaurant,Plaza,Café,Bakery,Indie Movie Theater
11,"Shyambazar, Burtolla, Hatibagan",Train Station,Park,Metro Station,Bengali Restaurant,Fast Food Restaurant,Plaza,Bakery,Mughlai Restaurant,Café,Pizza Place


<h3>Analyzing Cluster 4</h3>

In [56]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 4, kolkata_merged.columns[[0] + list(range(4, kolkata_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Ward no,,,,,,,,,,,
79,"Kidderpore (Babu Bazar), Mominpur, Majherhat, ...",Harbor / Marina,Awadhi Restaurant,ATM,Bakery,Food,Flea Market,Field,Fast Food Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant
97,"Ashok Nagar, Kudghat, Tollygunge Club, Regent ...",ATM,Plaza,Café,Dumpling Restaurant,Supermarket,Hotel,Lounge,Tex-Mex Restaurant,Golf Course,Movie Theater
116,Tollygunge Circular Road (Sirity-Senhati Colony),ATM,Mughlai Restaurant,Golf Course,Lounge,Movie Theater,Sports Club,Bakery,Plaza,Restaurant,Asian Restaurant
124,"Barisha, Thakurpukur",Pharmacy,ATM,Motorcycle Shop,Bakery,Electronics Store,Flea Market,Field,Fast Food Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant
127,Sarsuna (Sarsuna satellite township-Shakuntala...,ATM,Market,American Restaurant,Jewelry Store,Vegetarian / Vegan Restaurant,Falafel Restaurant,Food,Flea Market,Field,Fast Food Restaurant
131,Behala (Parnasree Pally),ATM,Mughlai Restaurant,Auto Dealership,Movie Theater,Vegetarian / Vegan Restaurant,Flea Market,Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store
137,Garden Reach (Metiabruz-Lichu Bagan),ATM,River,Botanical Garden,Bakery,Vegetarian / Vegan Restaurant,Electronics Store,Field,Fast Food Restaurant,Falafel Restaurant,Dumpling Restaurant
139,Metiabruz (Bartala-Karbala),Pizza Place,ATM,River,Bakery,Dumpling Restaurant,Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Donut Shop
142,Joka,ATM,Pharmacy,Motorcycle Shop,Health & Beauty Service,Hostel,Bus Station,Indian Sweet Shop,Pizza Place,Café,Bakery
